Практика 3: Задание аналогично заданию 2, но для представления документов используются вектора, которые показывают распределение тем коллекции в этом документе. По сути каждый признак - это тема, значение признака  в векторе - вероятность этой темы в этом документа. Для того. чтобы это сделать используем LDA. + Нужно посмотреть какие темы будут появлятся, что эти темы из себя представляют. Подсказка в файле

# LDA

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import math
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import scipy.sparse as sp
from tqdm.notebook import tqdm

In [2]:
import string
import nltk
from gensim.corpora import Dictionary
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim.models import LdaModel
import re

/Users/igorkopylov/PycharmProjects/liner_model/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
from sklearn.datasets import fetch_20newsgroups
number_of_topic = 3
newsgroups_train = fetch_20newsgroups(subset='train')
indices = np.where((newsgroups_train.target >= 0) & (newsgroups_train.target <= number_of_topic))[0]
texts = [newsgroups_train.data[i] for i in indices]
target = np.array([newsgroups_train.target[i] for i in indices])
print(f"Всего элементов: {target.shape[0]}")
for i in range(number_of_topic):
    count = np.count_nonzero(target == i)
    print(f"ко-во элементов из класса {i}: {count}")

Всего элементов: 2245
ко-во элементов из класса 0: 480
ко-во элементов из класса 1: 584
ко-во элементов из класса 2: 591


In [5]:
def lemmatize_tokens(tokens):
    lemmatizer = WordNetLemmatizer()

    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    return lemmatized_tokens


def preprocess_text(text): 
    # Удаление пунктуации
    text = re.sub(r'[^\w\s]', '', text)
    
    tokens = word_tokenize(text)

    stop_words = set(stopwords.words('english'))
    
    tokens = [token for token in tokens if token not in stop_words]

    lemmatized_tokens = lemmatize_tokens(tokens)
    
    return lemmatized_tokens

In [6]:
tokenized_corpus = [preprocess_text(i) for i in texts]
tokenized_corpus[0][:5]

['From', 'jgreenamber', 'Joe', 'Green', 'Subject']

## Подбор гиперпараметров

In [7]:
def lda_repo(tokenized_corpus, num_topics=15):
    # Создание словаря
    dictionary = Dictionary(tokenized_corpus)
    
    # Преобразование текстов в мешок слов
    bow_corpus = [dictionary.doc2bow(text) for text in tokenized_corpus]
    
    # Обучение модели LDA
    lda_model = LdaModel(
        corpus=bow_corpus,  # Мешок слов
        id2word=dictionary,  # Словарь
        num_topics=num_topics,  # Количество тем
        passes=10,  # Количество итераций по корпусу
        iterations=100,  # Количество итераций для оценки параметров
        random_state=42  # Зафиксированный случайный seed для воспроизводимости результатов
    )
    return lda_model, bow_corpus

In [8]:
classifier = DecisionTreeClassifier(max_depth=6)  
maxi_f1 = -1
count_of_sample = 100
best_param = []
for num_top in tqdm(range(1800, 2000, 10)):
    lda_model, bow_corpus = lda_repo(tokenized_corpus[:count_of_sample], num_top)
    lda_vec = []
    # получаем тематические векторы
    for i in range(count_of_sample):
        doc_topics = lda_model.get_document_topics(bow_corpus[i])
        doc_vec = np.zeros(num_top)
        for topic, prob in doc_topics:
            #top_numb = int(re.sub(r'[a-zA-Z]', '', topic))
            doc_vec[topic] = prob
        if len(doc_vec) != num_top:
            raise "jopa"
        lda_vec.append(doc_vec)
    X_train, X_test, y_train, y_test = train_test_split(lda_vec, target[:count_of_sample], test_size=0.2, random_state=42)
    
    classifier.fit(X_train, y_train)
    
    y_pred = classifier.predict(X_test)
    f1 = f1_score(y_test, y_pred, average='weighted')
    if f1 > maxi_f1:
        maxi_f1 = f1
        best_param = num_top
        

  0%|          | 0/20 [00:00<?, ?it/s]

In [9]:
best_param

1860

In [ ]:
lda_model, bow_corpus = lda_repo(tokenized_corpus, best_param)
lda_vec = []

In [88]:
# получаем тематические векторы
for i in tqdm(range(len(target))):
    doc_topics = lda_model.get_document_topics(bow_corpus[i])
    doc_vec = np.zeros(best_param)
    for topic, prob in doc_topics:
        #top_numb = int(re.sub(r'[a-zA-Z]', '', topic))
        doc_vec[topic] = prob
    lda_vec.append(doc_vec)

  0%|          | 0/2245 [00:00<?, ?it/s]

In [89]:
print(len(lda_vec), len(lda_vec[0]), len(target))

2245 1810 2245


In [90]:
for topic_id, topic_words in lda_model.show_topics()[:5]:
    print(f"Тема {topic_id}: {topic_words}")

Тема 1053: 0.000*"mail" + 0.000*"file" + 0.000*"image" + 0.000*"3D" + 0.000*"send" + 0.000*"ray" + 0.000*"The" + 0.000*"object" + 0.000*"format" + 0.000*"also"
Тема 782: 0.260*"VLB" + 0.120*"actually" + 0.095*"Have" + 0.094*"happens" + 0.064*"hook" + 0.057*"seen" + 0.046*"ever" + 0.038*"slows" + 0.037*"bus" + 0.031*"killed"
Тема 91: 0.000*"algebraically" + 0.000*"afil" + 0.000*"interection" + 0.000*"colinier" + 0.000*"colinear" + 0.000*"saturncsswinozau" + 0.000*"alansaturncsswinOZAU" + 0.000*"Repeat" + 0.000*"Swinburne" + 0.000*"7th"
Тема 1387: 0.000*"algebraically" + 0.000*"afil" + 0.000*"interection" + 0.000*"colinier" + 0.000*"colinear" + 0.000*"saturncsswinozau" + 0.000*"alansaturncsswinOZAU" + 0.000*"Repeat" + 0.000*"Swinburne" + 0.000*"7th"
Тема 1247: 0.000*"algebraically" + 0.000*"afil" + 0.000*"interection" + 0.000*"colinier" + 0.000*"colinear" + 0.000*"saturncsswinozau" + 0.000*"alansaturncsswinOZAU" + 0.000*"Repeat" + 0.000*"Swinburne" + 0.000*"7th"


# Обучение классификатора

In [83]:
classifier = DecisionTreeClassifier()  

X_train, X_test, y_train, y_test = train_test_split(lda_vec, target, test_size=0.2, random_state=42)

param_grid = {'max_depth': range(30, 100, 1)}

grid_search = GridSearchCV(classifier, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_

y_pred = best_model.predict(X_test)

f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Лучшая глубина дерева {grid_search.best_params_['max_depth']}")
print(f"значение F1-score:{f1}")

Лучшая глубина дерева 89
значение F1-score:0.6106153023551465
